<h1><b>Άσκηση 4</b></h1>
<p align="justify">Η μέθοδος Monte Carlo είναι μια υπολογιστική μέθοδος, που βασίζεται στο νόμο των μεγάλων αριθμών. Αν {Χ<sub>n</sub>}<sub>n∈N</sub> είναι μια ακολουθία από ανεξάρτητες, ισόνομες τυχαίες μεταβλητές, με πεπερασμένη μέση τιμή Ε[Χ], τότε:</p>

$$
P\left[
\frac{1}{n}\sum_{k=1}^{n}X_k \rightarrow E[X]
\right] = 1
$$

<p align="justify">Προκειμένου να υπολογίσουμε τη μέση τιμή Ε[Χ]  μιας τυχαίας μεταβλητής Χ, μπορούμε λοιπόν να πάρουμε το μέσο όρο ενός μεγάλου αριθμού ανεξάρτητων δειγμάτων αυτής της μεταβλητής. Με παρόμοιο τρόπο, μπορούμε να προσεγγίσουμε υπολογιστικά την πιθανότητα ενός ενδεχομένου από το κλάσμα των πραγματοποιήσεών του σε μια σειρά από <b>m</b> ανεξάρτητες προσομοιώσεις μέχρι το βήμα <b>n</b>, δηλαδή:</p>

$$
P\left[
\frac{1}{m} \sum_{k=1}^{m}H_k \rightarrow P[X_n | X_0]
\right] = 1
$$

<p align="justify">όπου η τυχαία μεταβλητή Η_k παίρνει την τιμή 1 εάν το ενδεχόμενο πραγματοποιείται στο τέλος του εκάστοτε πειράματος και 0 στην αντίθετη περίπτωση. Σ’ αυτήν την ιδέα θα βασιστεί η άσκηση αυτή. Σας δίνεται η μαρκοβιανή αλυσίδα στο χώρο καταστάσεων <b>Χ</b>={1,2,3} με πίνακα πιθανοτήτων μετάβασης:</p>

$$
P = \begin{pmatrix}
0 & 1 & 0\\
0 & 2/3 & 1/3\\
1/6 & 5/6 & 0
\end{pmatrix}
$$

<p align="justify">Χρησιμοποιώντας το πρόγραμμα που δίνεται παρακάτω, θα πραγματοποιήσετε <b>m</b> ανεξάρτητα πειράματα για να εκτιμήσετε την πιθανότητα <b>Για να τρέξετε το πρόγραμμα θα πρέπει να έχετε φορτώσει το αρχείο <i><a href="https://github.com/nkostopoulos/StochasticsLabPublic/blob/master/lab3/simple_markov_chain_lib.py">simple_markov_chain_lib.py</a></i></b>.</p>

$$
P\left[
X_{40} = 1 | X_0 = 1
\right]
$$

<p align="justify">δηλαδή την πιθανότητα να βρίσκεται η αλυσίδα στην κατάσταση 1 στο 40ό βήμα της δεδομένου ότι ξεκίνησε από την κατάσταση 1. Για να ελέγξετε την ορθότητα της μεθόδου, το πρόγραμμα περιλαμβάνει και τον ακριβή υπολογισμό της παραπάνω πιθανότητας.</p>
<ul>
<li>Να μελετήσετε το πρόγραμμα και να περιγράψετε σύντομα τη μέθοδο που ακολουθείται.</li>
<li>Να επαναλάβετε τη διαδικασία για τιμές της παραμέτρου m: (α) 1,000, (β) 10,000, (γ) 50,000, (δ) 100,000, (ε) 500,000. Να καταγράψετε και να σχολιάσετε την τιμή της παραπάνω πιθανότητας όπως υπολογίζεται από την προσομοίωση σε σχέση με την ακριβή τιμή της.</li>
</ul>


In [1]:
# This code is taken from the laboratory of "Stochastic Processes" course of ECE NTUA 6th semester, Prof. Loulakis
from bisect import bisect_left
from random import random

import networkx as nx  # get communication classes
import numpy as np
from scipy.sparse import csr_matrix


class markov_chain:

    def __init__(self, markov_table, init_dist=None):
        """
        Constructs a Markov Chain from a transition matrix.
        The initial distribution can be provided or setted aftewards.
        """

        # Attributes
        self.running_state = None
        self.steps = 0
        self.visits = {state: 0 for state in markov_table}
        size = len(markov_table)

        # Set up state transition probs
        self._states = {state: self._partial_sums(dist)
                        for state, dist in markov_table.items()}
        for state, dist in self._states.items():
            if not np.isclose(dist[-1][0], 1.0):
                msg = "State {} transitions do not add up to 1.0".format(state)
                raise ValueError(msg)
        self._probs_state = np.array([0] * size)

        # Adjacency Matrix
        data, rows, cols = [], [], []
        for row, dist in markov_table.items():
            col, pval = zip(*[(s, p) for s, p in dist.items() if p > 0])
            rows += [row] * len(col)
            cols += col
            data += pval
        # make sure they are in the right order
        enum = {state: i for i, state in enumerate(self._states)}
        rows = [enum[r] for r in rows]
        cols = [enum[c] for c in cols]
        self._adj = csr_matrix((data, (rows, cols)), shape=(size, size))

        # Communication Classes
        classes = {'Closed': [], 'Open': []}
        g = nx.MultiDiGraph(self._adj)
        scc = list(nx.strongly_connected_components(g))
        g = nx.condensation(g)  # SCCs collapse to single nodes
        for n in g:
            if g.out_degree(n) == 0:
                classes["Closed"].append(scc[n])
            else:
                classes["Open"].append(scc[n])
        self.communication_classes = classes

        # Set Initial State
        self._init_dist = None
        if init_dist is not None:
            self.init_dist = init_dist

    def __len__(self):
        """The cardinality of the state-space"""
        return len(self._states)

    @property
    def probs_matrix(self):
        """The transition probability matrix"""
        return self._adj.toarray()

    @property
    def probs_state(self):
        """
        Computes analytically the probability of being in every state at
        currentn step. Returns a vector of state probabilities
        """
        init_dist = np.array([self.init_dist.get(state, 0.0)
                              for state in self._states])
        probs = init_dist @ (self._adj ** self.steps)
        return dict(zip(self._states, probs))

    @property
    def init_dist(self):
        """The initial distribution of the chain"""
        return self._init_dist

    @init_dist.setter
    def init_dist(self, dist):
        if not np.isclose(sum(dist.values()), 1.0):
            msg = "The transition probabilities of init_dist must add up to 1.0"
            raise ValueError(msg)
        self._init_dist = dist
        self._state0 = self._partial_sums(dist)
        self.running_state = None

    @property
    def eigenvalues(self):
        """Returns the eigenvalues of the transition table"""
        return list(np.sort(np.linalg.eigvals(self.probs_matrix)))

    def _partial_sums(self, dist):
        """
        Takes as input a row of the probability matrix (dist)
        and generates its partial sums.
        These are cached as tuples (sum, state) to be sampled.
        """
        states, probs = zip(*[(s, p) for s, p in dist.items() if p > 0])
        probs = np.cumsum(probs)
        return list(zip(probs, states))

    def _next_state(self, state):
        """Selects a new state based on the transition probabilities"""
        return state[bisect_left(state, (random(), ))][1]

    def start(self):
        """First step of the chain choosen from the initial distribution"""

        # Initiate walk
        self.steps = 0
        for state in self._states:
            self.visits[state] = 0

        # Initialize the state distribution - to be updated as we walk
        self.running_state = self._next_state(self._state0)
        self.visits[self.running_state] = 1

    def move(self):
        """Moves to the next state and updates all relevant fields"""
        transition_probs = self._states[self.running_state]
        self.running_state = self._next_state(transition_probs)
        self.steps += 1
        self.visits[self.running_state] += 1

Τώρα εκτελούμε τον αλγόριθμο για experiments=500000

In [4]:
%%time
from __future__ import division
#from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

def defineMarkovTable():
	p = 1/6
	markov_table = {
		1: {2: 1.},
		2: {2: 2/3, 3: 1/3},
		3: {1: p, 2: 1-p}
	}

	return markov_table

def defineNumpyTable():
	Pn = np.array([[0,1,0],
		       [0,2/3,1/3],
		       [1/6, 5/6, 0]])
	P0 = np.array([[1,0,0]])

	return Pn,P0

def multiplyNumpyTables(Pn,P0):
	for index in range(40):
		Pn = np.matmul(Pn,Pn)
	Pn = np.matmul(P0,Pn)
	return Pn

def defineInitDistribution():
	init_dist = {1: 1.}

	return init_dist

def calculateProbabilities(markov_table, init_dist):
	mc = markov_chain(markov_table, init_dist)
	experiments = 500000
	steps = 40
	visits = 0

	for index in range(experiments):
		mc.start()
		for j in range(steps):
			mc.move()
		if mc.running_state == 1: visits += 1

	probability = visits / experiments
	return probability

if __name__ == "__main__":
	markov_table = defineMarkovTable()
	init_dist = defineInitDistribution()

	probability = calculateProbabilities(markov_table, init_dist)
	print(probability)

	Pn,P0 = defineNumpyTable()
	realProbability = multiplyNumpyTables(Pn,P0)
	print(realProbability)

0.039758
[[0.03999857 0.71997421 0.2399914 ]]
CPU times: user 19.3 s, sys: 54.2 ms, total: 19.4 s
Wall time: 21.6 s


ΣΧΟΛΙΟ:

Η πιθανότητα όπως υπολογίστηκε από την προσομοίωση των 500000 πειραμάτων είναι 0.039758.
Η θεωρητικά υπολογισμένη πιθανότητα είναι 0.03999857.
Άρα υπάρχει μία απόκλιση μεταξύ τους, της τάξης -0.6%.

ΑΠΑΝΤΗΣΕΙΣ:

i) Η μέθοδος Monte Carlo είναι μία τεχνική που χρησιμοποείται για την επίλυση προβλημάτων μέσω της προσομοίωσης τυχαίων διαδικασιών. Η μέθοδος αυτή χρησιμοποιεί την τυχαιότητα. Χρησιμοποιείται όταν δεν είναι δυνατή η επίλυση ενός προβλήατος με αναλυτικές μεθόδους ή όταν οι λύσεις είναι περίπλοκες. Η βασική ιδέα είναι να προσομοιώνονται τυχαίες διαδικασίες, πολλές φορές και στη συνέχεια να υπολογίζονται στατιστικά χαρακτηριστικά.

Η μέθοδος Monte Carlo χρησιμοποιείται για την βελτιστοποίηση των δεδομένων. Προσφέρει ταχύτητα στην εκτέλεση των πράξεων και είναι αμερόληπτη και συνεπής.

Σε σχέση τώρα με τον κώδικα της άσκησης, πραγματοποιείται ένα αριθμός πειραμάτων, ο οποίος ιδανικά πρέπει να είναι αρκετά υψηλός προκειμένου να προσεγγιστεί το όριο m -> 00 (άπειρο) με σχετικά μικρό σφάλμα, ώστε να υπάρχει υψηλή εμπιστοσύνη στη στατιστικά υπολογισμένη πιθανότητα. Σε κάθε πείραμα, ξεκινώντας από την κατάσταση 1, πραγματοποιούνται διαδοχικές μεταβάσεις από κατάσταση σε κατάσταση βάσει των σχετικών πιθανοτήτων μετάβασης, διαδικασία η οποία υλοποιείται μέσω παραγωγής ψευδοτυχαίων αριθμών. Το πείραμα ολοκληρώνεται έπειτα από 40 μεταβάσεις και στην περίπτωση που η τελική κατάσταση είναι η κατάσταση 1, μια δείκτρια μεταβλητή αυξάνεται κατά μία μονάδα. Η τελική πιθανότητα να βρισκόμαστε στην κατάσταση 1, έχοντας ξεκινήσει από αυτήν, μετά από 40 βήματα υπολογίζεται απλώς ως το πηλίκο της τιμής που έχει λάβει τελικά η δείκτρια μεταβλητή, προς το συνολικό αριθμό πειραμάτων. Σε ό,τι αφορά την αναλυτική προσέγγιση, το διάνυσμα της αρχικής κατάστασης Πο=(1,0,0)
πολλαπλασιάζεται με τον πίνακα Ρ40, ώστε να προκύψει το διάνυσμα της αναμενόμενης κατανομής καταστάσεων μετά από 40 βήματα. Η τιμή της πρώτης συνιστώσας του διανύσματος αυτού αντιστοιχεί στη ζητούμενη πιθανότητα.

ii)

Τώρα εκτελούμε τον αλγόριθμο για experiments=1000

In [3]:
%%time
from __future__ import division
#from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

def defineMarkovTable():
	p = 1/6
	markov_table = {
		1: {2: 1.},
		2: {2: 2/3, 3: 1/3},
		3: {1: p, 2: 1-p}
	}

	return markov_table

def defineNumpyTable():
	Pn = np.array([[0,1,0],
		       [0,2/3,1/3],
		       [1/6, 5/6, 0]])
	P0 = np.array([[1,0,0]])

	return Pn,P0

def multiplyNumpyTables(Pn,P0):
	for index in range(40):
		Pn = np.matmul(Pn,Pn)
	Pn = np.matmul(P0,Pn)
	return Pn

def defineInitDistribution():
	init_dist = {1: 1.}

	return init_dist

def calculateProbabilities(markov_table, init_dist):
	mc = markov_chain(markov_table, init_dist)
	experiments = 1000
	steps = 40
	visits = 0

	for index in range(experiments):
		mc.start()
		for j in range(steps):
			mc.move()
		if mc.running_state == 1: visits += 1

	probability = visits / experiments
	return probability

if __name__ == "__main__":
	markov_table = defineMarkovTable()
	init_dist = defineInitDistribution()

	probability = calculateProbabilities(markov_table, init_dist)
	print(probability)

	Pn,P0 = defineNumpyTable()
	realProbability = multiplyNumpyTables(Pn,P0)
	print(realProbability)

0.035
[[0.03999857 0.71997421 0.2399914 ]]
CPU times: user 67.2 ms, sys: 1.99 ms, total: 69.2 ms
Wall time: 80.4 ms


ΣΧΟΛΙΟ:

Η πιθανότητα όπως υπολογίστηκε από την προσομοίωση των 1000 πειραμάτων είναι
 0.035. Η θεωρητικά υπολογισμένη πιθανότητα είναι 0.03999857. Δηλαδή 0.04. Άρα υπάρχει μία απόκλιση μεταξύ τους, της τάξης -12.5%.

Τώρα εκτελούμε τον αλγόριθμο για experiments=10000

In [5]:
%%time
from __future__ import division
#from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

def defineMarkovTable():
	p = 1/6
	markov_table = {
		1: {2: 1.},
		2: {2: 2/3, 3: 1/3},
		3: {1: p, 2: 1-p}
	}

	return markov_table

def defineNumpyTable():
	Pn = np.array([[0,1,0],
		       [0,2/3,1/3],
		       [1/6, 5/6, 0]])
	P0 = np.array([[1,0,0]])

	return Pn,P0

def multiplyNumpyTables(Pn,P0):
	for index in range(40):
		Pn = np.matmul(Pn,Pn)
	Pn = np.matmul(P0,Pn)
	return Pn

def defineInitDistribution():
	init_dist = {1: 1.}

	return init_dist

def calculateProbabilities(markov_table, init_dist):
	mc = markov_chain(markov_table, init_dist)
	experiments = 10000
	steps = 40
	visits = 0

	for index in range(experiments):
		mc.start()
		for j in range(steps):
			mc.move()
		if mc.running_state == 1: visits += 1

	probability = visits / experiments
	return probability

if __name__ == "__main__":
	markov_table = defineMarkovTable()
	init_dist = defineInitDistribution()

	probability = calculateProbabilities(markov_table, init_dist)
	print(probability)

	Pn,P0 = defineNumpyTable()
	realProbability = multiplyNumpyTables(Pn,P0)
	print(realProbability)

0.0409
[[0.03999857 0.71997421 0.2399914 ]]
CPU times: user 605 ms, sys: 1.23 ms, total: 606 ms
Wall time: 634 ms


ΣΧΟΛΙΟ:

Η πιθανότητα όπως υπολογίστηκε από την προσομοίωση των 10000 πειραμάτων είναι
 0.0409. Η θεωρητικά υπολογισμένη πιθανότητα είναι 0.03999857. Δηλαδή 0.04. Άρα υπάρχει μία απόκλιση μεταξύ τους, της τάξης 2.25%.

Τώρα εκτελούμε τον αλγόριθμο για experiments=50000

In [7]:
%%time
from __future__ import division
#from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

def defineMarkovTable():
	p = 1/6
	markov_table = {
		1: {2: 1.},
		2: {2: 2/3, 3: 1/3},
		3: {1: p, 2: 1-p}
	}

	return markov_table

def defineNumpyTable():
	Pn = np.array([[0,1,0],
		       [0,2/3,1/3],
		       [1/6, 5/6, 0]])
	P0 = np.array([[1,0,0]])

	return Pn,P0

def multiplyNumpyTables(Pn,P0):
	for index in range(40):
		Pn = np.matmul(Pn,Pn)
	Pn = np.matmul(P0,Pn)
	return Pn

def defineInitDistribution():
	init_dist = {1: 1.}

	return init_dist

def calculateProbabilities(markov_table, init_dist):
	mc = markov_chain(markov_table, init_dist)
	experiments = 50000
	steps = 40
	visits = 0

	for index in range(experiments):
		mc.start()
		for j in range(steps):
			mc.move()
		if mc.running_state == 1: visits += 1

	probability = visits / experiments
	return probability

if __name__ == "__main__":
	markov_table = defineMarkovTable()
	init_dist = defineInitDistribution()

	probability = calculateProbabilities(markov_table, init_dist)
	print(probability)

	Pn,P0 = defineNumpyTable()
	realProbability = multiplyNumpyTables(Pn,P0)
	print(realProbability)

0.0404
[[0.03999857 0.71997421 0.2399914 ]]
CPU times: user 1.76 s, sys: 8.97 ms, total: 1.77 s
Wall time: 1.78 s


ΣΧΟΛΙΟ:

Η πιθανότητα όπως υπολογίστηκε από την προσομοίωση των 50000 πειραμάτων είναι 0.0404. Η θεωρητικά υπολογισμένη πιθανότητα είναι 0.03999857. Δηλαδή 0.04. Άρα υπάρχει μία απόκλιση μεταξύ τους, της τάξης 1%.

Τώρα εκτελούμε τον αλγόριθμο για experiments=100000

In [8]:
%%time
from __future__ import division
#from simple_markov_chain_lib import markov_chain
import statistics as stat
from numpy import matmul
import numpy as np

def defineMarkovTable():
	p = 1/6
	markov_table = {
		1: {2: 1.},
		2: {2: 2/3, 3: 1/3},
		3: {1: p, 2: 1-p}
	}

	return markov_table

def defineNumpyTable():
	Pn = np.array([[0,1,0],
		       [0,2/3,1/3],
		       [1/6, 5/6, 0]])
	P0 = np.array([[1,0,0]])

	return Pn,P0

def multiplyNumpyTables(Pn,P0):
	for index in range(40):
		Pn = np.matmul(Pn,Pn)
	Pn = np.matmul(P0,Pn)
	return Pn

def defineInitDistribution():
	init_dist = {1: 1.}

	return init_dist

def calculateProbabilities(markov_table, init_dist):
	mc = markov_chain(markov_table, init_dist)
	experiments = 100000
	steps = 40
	visits = 0

	for index in range(experiments):
		mc.start()
		for j in range(steps):
			mc.move()
		if mc.running_state == 1: visits += 1

	probability = visits / experiments
	return probability

if __name__ == "__main__":
	markov_table = defineMarkovTable()
	init_dist = defineInitDistribution()

	probability = calculateProbabilities(markov_table, init_dist)
	print(probability)

	Pn,P0 = defineNumpyTable()
	realProbability = multiplyNumpyTables(Pn,P0)
	print(realProbability)

0.03953
[[0.03999857 0.71997421 0.2399914 ]]
CPU times: user 5.46 s, sys: 16.4 ms, total: 5.48 s
Wall time: 7.14 s


ΣΧΟΛΙΟ:

Η πιθανότητα όπως υπολογίστηκε από την προσομοίωση των 100000 πειραμάτων είναι 0.039758. Η θεωρητικά υπολογισμένη πιθανότητα είναι 0.03999857. Δηλαδή 0.04. Άρα υπάρχει μία απόκλιση μεταξύ τους, της τάξης -1.17%.